# Pony examples

### Diagram of Database for example

![](eStore.png)


In [1]:
from pony.orm.examples.estore import *

GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
SELECT "CartItem"."id", "CartItem"."quantity", "CartItem"."customer", "CartItem"."product"
FROM "CartItem" "CartItem"
WHERE 0 = 1

SELECT "Category"."id", "Category"."name"
FROM "Category" "Category"
WHERE 0 = 1

SELECT "Category_Product"."category", "Category_Product"."product"
FROM "Category_Product" "Category_Product"
WHERE 0 = 1

SELECT "Customer"."id", "Customer"."email", "Customer"."password", "Customer"."name", "Customer"."country", "Customer"."address"
FROM "Customer" "Customer"
WHERE 0 = 1

SELECT "Order"."id", "Order"."state", "Order"."date_created", "Order"."date_shipped", "Order"."date_delivered", "Order"."total_price", "Order"."customer"
FROM "Order" "Order"
WHERE 0 = 1

SELECT "OrderItem"."quantity", "OrderItem"."price", "OrderItem"."order", "OrderItem"."product"
FROM "OrderItem" "OrderItem"
WHERE 0 = 1

SELECT "Product"."id", "Product"."name", "Product"."description", "Product"."pi

In [2]:
with db_session:
    if Customer.select().first() is None:
        populate_database()

GET NEW CONNECTION
SWITCH TO AUTOCOMMIT MODE
SELECT "c"."id", "c"."email", "c"."password", "c"."name", "c"."country", "c"."address"
FROM "Customer" "c"
ORDER BY 1
LIMIT 1

RELEASE CONNECTION


## All USA customers


In [3]:
usa_customers = select(customer for customer in Customer
                       if customer.country == 'USA')
show(usa_customers)

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "customer"."id", "customer"."email", "customer"."password", "customer"."name", "customer"."country", "customer"."address"
FROM "Customer" "customer"
WHERE "customer"."country" = 'USA'

id|email              |password|name          |country|address  
--+-------------------+--------+--------------+-------+---------
1 |john@example.com   |***     |John Smith    |USA    |address 1
2 |matthew@example.com|***     |Matthew Reed  |USA    |address 2
4 |rebecca@example.com|***     |Rebecca Lawson|USA    |address 4


## The number of customers for each country


In [4]:
customers_each_country = select(
    (customer.country, count(customer)) for customer in Customer)
show(customers_each_country)

SELECT "customer"."country", COUNT(DISTINCT "customer"."id")
FROM "Customer" "customer"
GROUP BY "customer"."country"

customer.country|count(customer)
----------------+---------------
China           |1              
UK              |1              
USA             |3              


## Max product price


In [5]:
max_product_price = max(product.price for product in Product)
show(max_product_price)

SELECT MAX("product"."price")
FROM "Product" "product"

Decimal('478.5')


In [6]:
max_ssd_price = max(product.price for product in Product
                    for category in product.categories
                    if category.name == 'Solid State Drives')
show(max_ssd_price)

SELECT MAX("product"."price")
FROM "Product" "product", "Category_Product" "t-1", "Category" "category"
WHERE "category"."name" = 'Solid State Drives'
  AND "product"."id" = "t-1"."product"
  AND "t-1"."category" = "category"."id"

Decimal('188.67')


## Three most expensive products


In [7]:
most_expensive_products = select(product for product in Product).order_by(
    desc(Product.price))[:3]
show(most_expensive_products)

SELECT "product"."id", "product"."name", "product"."description", "product"."picture", "product"."price", "product"."quantity"
FROM "Product" "product"
ORDER BY "product"."price" DESC
LIMIT 3

id|name                      |description               |picture|price |quantity
--+--------------------------+--------------------------+-------+------+--------
2 |Apple iPad with Retina ...|iPad with Retina displa...|None   |478.50|180     
1 |Kindle Fire HD            |Amazon tablet for web, ...|None   |284.00|120     
6 |Crucial m4 256GB 2.5-In...|The award-winning SSD d...|None   |188.67|60      


## Out of stock products


In [8]:
products_out_stock = select(product for product in Product
                            if product.quantity == 0)
show(products_out_stock)

SELECT "product"."id", "product"."name", "product"."description", "product"."picture", "product"."price", "product"."quantity"
FROM "Product" "product"
WHERE "product"."quantity" = 0

id|name                      |description               |picture|price|quantity
--+--------------------------+--------------------------+-------+-----+--------
5 |Samsung 840 Series 2.5 ...|Enables you to boot up ...|None   |98.95|0       


## Most popular product


In [9]:
most_popular_product = select(product for product in Product).order_by(
    lambda product: desc(sum(product.order_items.quantity)))[:1]
show(most_popular_product)

SELECT "product"."id"
FROM "Product" "product"
  LEFT JOIN "OrderItem" "orderitem"
    ON "product"."id" = "orderitem"."product"
GROUP BY "product"."id"
ORDER BY coalesce(SUM("orderitem"."quantity"), 0) DESC
LIMIT 1

SELECT "id", "name", "description", "picture", "price", "quantity"
FROM "Product"
WHERE "id" = ?
[4]

id|name                      |description               |picture|price|quantity
--+--------------------------+--------------------------+-------+-----+--------
4 |Kingston Digital DataTr...|Convenient - small, cap...|None   |9.98 |350     


In [10]:
## Products that have never been ordered

In [11]:
never_ordered_products = select(product for product in Product
                                if not product.order_items)
show(never_ordered_products)

SELECT "product"."id", "product"."name", "product"."description", "product"."picture", "product"."price", "product"."quantity"
FROM "Product" "product"
WHERE NOT EXISTS (
    SELECT 1
    FROM "OrderItem" "orderitem"
    WHERE "product"."id" = "orderitem"."product"
    )

id|name                      |description               |picture|price|quantity
--+--------------------------+--------------------------+-------+-----+--------
3 |SanDisk Cruzer 16 GB US...|Take it all with you on...|None   |9.99 |400     
5 |Samsung 840 Series 2.5 ...|Enables you to boot up ...|None   |98.95|0       


## Customers who made several orders


In [12]:
customers_several_orders = select(customer for customer in Customer
                                  if count(customer.orders) > 1)
show(customers_several_orders)

SELECT "customer"."id"
FROM "Customer" "customer"
  LEFT JOIN "Order" "order"
    ON "customer"."id" = "order"."customer"
GROUP BY "customer"."id"
HAVING COUNT(DISTINCT "order"."id") > 1

id|email           |password|name      |country|address  
--+----------------+--------+----------+-------+---------
1 |john@example.com|***     |John Smith|USA    |address 1


## Three most valuable customers


In [13]:
most_valuable_customer = select(customer for customer in Customer).order_by(
    lambda customer: desc(sum(customer.orders.total_price)))[:3]
show(most_valuable_customer)

SELECT "customer"."id"
FROM "Customer" "customer"
  LEFT JOIN "Order" "order"
    ON "customer"."id" = "order"."customer"
GROUP BY "customer"."id"
ORDER BY coalesce(SUM("order"."total_price"), 0) DESC
LIMIT 3

id|email              |password|name          |country|address  
--+-------------------+--------+--------------+-------+---------
1 |john@example.com   |***     |John Smith    |USA    |address 1
4 |rebecca@example.com|***     |Rebecca Lawson|USA    |address 4
2 |matthew@example.com|***     |Matthew Reed  |USA    |address 2


## Customers whose orders were shipped


In [14]:
customers_order_shipped = select(customer for customer in Customer
                                 if SHIPPED in customer.orders.state)
show(customers_order_shipped)

SELECT "customer"."id", "customer"."email", "customer"."password", "customer"."name", "customer"."country", "customer"."address"
FROM "Customer" "customer"
WHERE ? IN (
    SELECT "order"."state"
    FROM "Order" "order"
    WHERE "customer"."id" = "order"."customer"
    )
['SHIPPED']

id|email               |password|name     |country|address  
--+--------------------+--------+---------+-------+---------
3 |chuanqin@example.com|***     |Chuan Qin|China  |address 3


In [15]:
# The same query with the INNER JOIN instead of IN.
customers_order_shipped_join = select(
    customer for customer in Customer
    if JOIN(SHIPPED in customer.orders.state))
show(customers_order_shipped_join)

SELECT DISTINCT "customer"."id", "customer"."email", "customer"."password", "customer"."name", "customer"."country", "customer"."address"
FROM "Customer" "customer", "Order" "order"
WHERE "order"."state" = ?
  AND "customer"."id" = "order"."customer"
['SHIPPED']

id|email               |password|name     |country|address  
--+--------------------+--------+---------+-------+---------
3 |chuanqin@example.com|***     |Chuan Qin|China  |address 3


## Customers with no orders


In [16]:
customers_no_order = select(customer for customer in Customer
                            if not customer.orders)
show(customers_no_order)

SELECT "customer"."id", "customer"."email", "customer"."password", "customer"."name", "customer"."country", "customer"."address"
FROM "Customer" "customer"
WHERE NOT EXISTS (
    SELECT 1
    FROM "Order" "order"
    WHERE "customer"."id" = "order"."customer"
    )

id|email             |password|name         |country|address  
--+------------------+--------+-------------+-------+---------
5 |oliver@example.com|***     |Oliver Blakey|UK     |address 5


In [17]:
# The same query with the LEFT JOIN instead of NOT EXISTS.
customers_no_order_left_join = left_join(customer for customer in Customer
                                         for order in customer.orders
                                         if order is None)
show(customers_no_order_left_join)

SELECT DISTINCT "customer"."id", "customer"."email", "customer"."password", "customer"."name", "customer"."country", "customer"."address"
FROM "Customer" "customer"
  LEFT JOIN "Order" "order"
    ON "customer"."id" = "order"."customer"
WHERE "order"."id" IS NULL

id|email             |password|name         |country|address  
--+------------------+--------+-------------+-------+---------
5 |oliver@example.com|***     |Oliver Blakey|UK     |address 5


## Customers which ordered several products from the same category'


In [18]:
customers_ordered_products_same_category = select(
    (customer.name, category.name) for customer in Customer
    for product in customer.orders.items.product
    for category in product.categories if count(product) > 1)
show(customers_ordered_products_same_category)

SELECT "customer"."name", "category"."name"
FROM "Customer" "customer", "Order" "order", "OrderItem" "orderitem", "Category_Product" "t-1", "Category" "category"
WHERE "customer"."id" = "order"."customer"
  AND "order"."id" = "orderitem"."order"
  AND "orderitem"."product" = "t-1"."product"
  AND "t-1"."category" = "category"."id"
GROUP BY "customer"."name", "category"."name"
HAVING COUNT(DISTINCT "orderitem"."product") > 1

customer.name |category.name
--------------+-------------
John Smith    |Tablets      
Matthew Reed  |Data Storage 
Rebecca Lawson|Tablets      


## Products whose price varies over time


In [19]:
products_price_varies = select(product for product in Product
                               if count(product.order_items.price) > 1)
show(products_price_varies)

SELECT "product"."id"
FROM "Product" "product"
  LEFT JOIN "OrderItem" "orderitem"
    ON "product"."id" = "orderitem"."product"
GROUP BY "product"."id"
HAVING COUNT(DISTINCT "orderitem"."price") > 1

id|name          |description                           |picture|price |quantity
--+--------------+--------------------------------------+-------+------+--------
1 |Kindle Fire HD|Amazon tablet for web, movies, musi...|None   |284.00|120     


In [20]:
# The same query, but with min and max price for each product.
products_price_varies_min_max = select(
    (product.name, min(product.order_items.price),
     max(product.order_items.price)) for product in Product
    if count(product.order_items.price) > 1)
show(products_price_varies_min_max)

SELECT "product"."name", MIN("orderitem"."price"), MAX("orderitem"."price")
FROM "Product" "product"
  LEFT JOIN "OrderItem" "orderitem"
    ON "product"."id" = "orderitem"."product"
GROUP BY "product"."name"
HAVING COUNT(DISTINCT "orderitem"."price") > 1

product.name  |min(product.order_items.price)|max(product.order_items.price)
--------------+------------------------------+------------------------------
Kindle Fire HD|274                           |284                           


## Orders with a discount

order total price < sum of order item prices


In [21]:
orders_with_discount = select(
    order for order in Order
    if order.total_price < sum(order.items.price * order.items.quantity))
show(orders_with_discount)

SELECT "order"."id"
FROM "Order" "order"
  LEFT JOIN "OrderItem" "orderitem"
    ON "order"."id" = "orderitem"."order"
GROUP BY "order"."id"
HAVING "order"."total_price" < coalesce(SUM(("orderitem"."price" * "orderitem"."quantity")), 0)

SELECT "id", "state", "date_created", "date_shipped", "date_delivered", "total_price", "customer"
FROM "Order"
WHERE "id" IN (?, ?, ?, ?)
[1, 3, 4, 5]

id|state    |date_created |date_shipped |date_deliv...|total_price|customer   
--+---------+-------------+-------------+-------------+-----------+-----------
1 |DELIVERED|2012-10-20...|2012-10-21...|2012-10-26...|292.00     |Customer[1]
3 |DELIVERED|2012-11-03...|2012-11-04...|2012-11-07...|680.50     |Customer[2]
4 |SHIPPED  |2013-03-11...|2013-03-12...|None         |99.80      |Customer[3]
5 |CREATED  |2013-03-15...|None         |None         |722.00     |Customer[4]
